In [1]:
import ee
%matplotlib inline
import math, datetime
import shapely
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

import datetime, calendar

plt.style.use("seaborn-darkgrid")

#ee.Authenticate()
ee.Initialize()

C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\theodore.wong\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [102]:
MODEL_INFO = {'UKESM1-0-LL': 'HadAM',
 'NorESM2-MM': 'CCM',
 'NorESM2-LM': 'CCM',
 'MRI-ESM2-0': 'UCLA GCM',
 'MPI-ESM1-2-LR': 'ECMWF',
 'MPI-ESM1-2-HR': 'ECMWF',
 'MIROC6': 'MIROC',
 'MIROC-ES2L': 'MIROC',
 'KIOST-ESM': 'GFDL',
 'KACE-1-0-G': 'HadAM',
 'IPSL-CM6A-LR': 'IPSL',
 'INM-CM5-0': 'INM',
 'INM-CM4-8': 'INM',
 'HadGEM3-GC31-MM': 'HadAM',
 'HadGEM3-GC31-LL': 'HadAM',
 'GFDL-ESM4': 'GFDL',
 'GFDL-CM4_gr2': 'GFDL',
 'GFDL-CM4': 'GFDL',
 'FGOALS-g3': 'CCM',
 'EC-Earth3-Veg-LR': 'ECMWF',
 'EC-Earth3': 'ECMWF',
 'CanESM5': 'CanAM',
 'CNRM-ESM2-1': 'ECMWF',
 'CNRM-CM6-1': 'ECMWF',
 'CMCC-ESM2': 'CCM',
 'CMCC-CM2-SR5': 'CCM',
 'BCC-CSM2-MR': 'CCM',
 'ACCESS-ESM1-5': 'HadAM',
 'ACCESS-CM2': 'HadAM',
 'TaiESM1': 'CCM',
}

EXCLUDED_MODELS = ['GFDL-CM4_gr2','ERA5'] 

MODELS = [i for i in MODEL_INFO if not i in EXCLUDED_MODELS]


HIST_START = 1980
HIST_END = 2014

NUM_BEST_MODELS = 5

flip_year = False

In [8]:
VARIABLES = {
    'tas': {
        'era_name': 'mean_2m_air_temperature',
        'era_transform': lambda x: x - 273.15,
        'nex_transform': lambda x: x - 273.15
    },
    'tasmax': {
        'era_name': 'maximum_2m_air_temperature',
        'era_transform': lambda x: x - 273.15,
        'nex_transform': lambda x: x - 273.15
    },
    'tasmin': {
        'era_name': 'minimum_2m_air_temperature',
        'era_transform': lambda x: x - 273.15,
        'nex_transform': lambda x: x - 273.15
    },
    'pr': {
        'era_name': 'total_precipitation',
        'era_transform': lambda x: x * 1000,
        'nex_transform': lambda x: x * 86400
    }
}

In [77]:
def d2j(datestring):
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday

In [137]:
def get_var(varname, model, latlon, start_year=HIST_START, end_year=HIST_END, yearshift=False, scenario='ssp585'):
    def removeLeapDays(arr, yearshift=False):
        if not yearshift:
            indices = []
            jan1_idx = 0
            for year in range(start_year, end_year+1):
                indices += [jan1_idx + i for i in range(365)]
                jan1_idx += 365
                if calendar.isleap(year):
                    jan1_idx += 1
            return arr[indices]
        else:
            indices = []
            jul1_idx = 0
            for year in range(start_year-1, end_year):
                indices += [jul1_idx + i for i in range(183)]
                jul1_idx += 183
                if calendar.isleap(year):
                    jul1_idx += 1
                indices += [jul1_idx + i for i in range(182)]
                jul1_idx += 182
            return arr[indices]
    if model != 'ERA5' and start_year < 2015 and end_year >= 2015:
        raise Exception("Requesting hist and non-hist variables in one query")
    if model == 'ERA5':
        dataset = ee.ImageCollection("ECMWF/ERA5/DAILY")
    else:
        dataset = ee.ImageCollection('NASA/GDDP-CMIP6').filter(ee.Filter.eq('model', model)).filter(ee.Filter.eq('scenario', [scenario, 'historical'][int(end_year<2015)]))
    gee_geom = ee.Geometry.Point((latlon[1], latlon[0]))
    select_varname = [varname, VARIABLES[varname]['era_name']][int(model == 'ERA5')]
    if not yearshift:
        data_vars = dataset.select(select_varname).filter(ee.Filter.date('{0}-01-01'.format(start_year), '{0}-01-01'.format(end_year+ 1)))
        result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        return [VARIABLES[varname]['nex_transform'], VARIABLES[varname]['era_transform']][int(model == 'ERA5')](removeLeapDays(np.array(result), False))
    else:
        data_vars = dataset.select(select_varname).filter(ee.Filter.date('{0}-07-01'.format(start_year-1), '{0}-07-01'.format(end_year)))
        result = [i[4] for i in data_vars.getRegion(gee_geom, 2500, 'epsg:4326').getInfo()[1:]]
        return [VARIABLES[varname]['nex_transform'], VARIABLES[varname]['era_transform']][int(model == 'ERA5')](removeLeapDays(np.array(result), True))

In [5]:
def get_rmsd(d1, d2):
    def seasonal_means(d):
        mam = []  # 60-151
        jja = []  # 152-243
        son = []  # 244-334
        djf = []  # 335-59
        jan1_idx = 365 + [0, 1][int(calendar.isleap(HIST_START))]
        for year in range(HIST_START+1, HIST_END):
            mam.append(d[jan1_idx + 60 : jan1_idx + 152])
            jja.append(d[jan1_idx + 152 : jan1_idx + 244])
            son.append(d[jan1_idx + 244 : jan1_idx + 335])
            if year < HIST_END - 1:
                if False and calendar.isleap(year):
                    yearlength = 366
                else:
                    yearlength = 365
                djf.append(np.concatenate((d[jan1_idx + 335 : jan1_idx + 365], d[jan1_idx + yearlength : jan1_idx + yearlength + 60])))
            else:
                djf.append(np.concatenate((d[335 : 365], d[365 + [0, 1][int(False and calendar.isleap(HIST_START))] : 425])))
            jan1_idx += 365 + [0, 1][int(False and calendar.isleap(year))]
        return np.array([np.mean(mam, axis=1), np.mean(jja, axis=1), np.mean(son, axis=1), np.mean(djf, axis=1)]).flatten()
            
    c1 = seasonal_means(d1)
    c2 = seasonal_means(d2)
    return np.sqrt(np.mean(np.sum((c1 - c2)**2)))

In [6]:
def quarters(d, start_year, end_year):
    mam = []  # 60-151
    jja = []  # 152-243
    son = []  # 244-334
    djf = []  # 335-59
    jan1_idx = 365# + [0, 1][int(calendar.isleap(start_year))]
    for year in range(start_year, end_year):
        tmp = np.concatenate((d[jan1_idx - 365 : jan1_idx - 365 + 60], d[jan1_idx + 335 : jan1_idx + 365]), axis=0)
        djf.append(tmp)
        mam.append(d[jan1_idx + 60 : jan1_idx + 152])
        jja.append(d[jan1_idx + 152 : jan1_idx + 244])
        son.append(d[jan1_idx + 244 : jan1_idx + 335])

        jan1_idx += 365 + [0, 0][int(False and calendar.isleap(year))]
    mam_res = np.vstack(mam)
    jja_res = np.vstack(jja)
    son_res = np.vstack(son)
    djf_res = np.vstack(djf)
    return mam_res, jja_res, son_res, djf_res
    
def seasonal_means(d):
    q = quarters(d, HIST_START, HIST_END)
    return np.array([np.mean(q[0], axis=1), np.mean(q[1], axis=1), np.mean(q[2], axis=1), np.mean(q[3], axis=1)])

In [7]:
def calibration_function(hist_obs, hist_mod):
    source = np.sort(hist_obs.flatten())
    target= np.sort(hist_mod.flatten())
   
    if (np.max(source) == 0 and np.min(source) == 0):
        return np.arange(0, target.size) / target.size
    if (np.max(target) == 0 and np.min(target) == 0):
        return np.arange(0, source.size) / source.size
    new_indices = []
    #source[-1] = target[-1]  # when target[i] greater than all source values, return max index
    for target_idx, target_value in enumerate(target):
        if target_idx < len(source):
            source_value = source[target_idx]
            if source_value > target[-1]:
                new_indices.append(target.size - 1)
            else:
                new_indices.append(np.argmax(target >= source_value))
    return np.array(new_indices) / source.size

def calibrate_component(uncalibrated_data, calibration_fxn):
    N = len(uncalibrated_data)
    unsorted_uncalib = [(i, idx) for idx, i in enumerate(uncalibrated_data)]
    sorted_uncalib = sorted(unsorted_uncalib)
    result = [0] * N
    for j in range(N):
        X_j = j / (N + 1)
        Y_jprime = calibration_fxn[math.floor(X_j * len(calibration_fxn))]
        jprime = math.floor(Y_jprime * (N + 1))
        result[sorted_uncalib[j][1]] = sorted_uncalib[min(len(sorted_uncalib)-1, jprime)][0]
    
    return result

def calibrate(uncalibrated_data, calibration_fxn):
    mam = []
    jja = []
    son = []
    djf = []
    mam_idx = []
    jja_idx = []
    son_idx = []
    djf_idx = []
    for idx, i in enumerate(uncalibrated_data):
        if idx % 365 >= 60 and idx % 365 < 152:
            mam.append(uncalibrated_data[idx])
            mam_idx.append(idx)
        elif idx % 365 >= 152 and idx % 365 < 244:
            jja.append(uncalibrated_data[idx])
            jja_idx.append(idx)
        elif idx % 365 >= 244 and idx % 365 < 335:
            son.append(uncalibrated_data[idx])
            son_idx.append(idx)
        else:
            djf.append(uncalibrated_data[idx])
            djf_idx.append(idx)
    
    mam_calib = calibrate_component(np.array(mam), calibration_fxn[0])
    jja_calib = calibrate_component(np.array(jja), calibration_fxn[1])
    son_calib = calibrate_component(np.array(son), calibration_fxn[2])
    djf_calib = calibrate_component(np.array(djf), calibration_fxn[3])
    
    result = [0] * len(uncalibrated_data)
    for i in range(len(mam_idx)):
        result[mam_idx[i]] = mam_calib[i]
    for i in range(len(jja_idx)):
        result[jja_idx[i]] = jja_calib[i]
    for i in range(len(son_idx)):
        result[son_idx[i]] = son_calib[i]
    for i in range(len(djf_idx)):
        result[djf_idx[i]] = djf_calib[i]

    return np.array(result)

In [33]:
def get_bestmodels(varname, latlon):
# Select three best models based on RMSD of quarterly mean tasmax
    hist_obs = get_var(varname, 'ERA5', latlon)
    hist_mods = {}
    rmsds = []
    for model in MODELS:
        hist_mod = get_var(varname, model, latlon, start_year=HIST_START-1)
        hist_mods[model] = hist_mod
        rmsds.append((get_rmsd(hist_obs, hist_mod), model))
    rmsds.sort()
    best_models = []
    families = []
    idx = 0
    while len(best_models) < NUM_BEST_MODELS:
        if not MODEL_INFO[rmsds[idx][1]] in families:
            best_models.append(rmsds[idx][1])
            families.append(MODEL_INFO[rmsds[idx][1]])
        idx += 1
    return best_models

In [11]:
latlons = {}
with open('latlon_Tamilnadu_Andhra.csv', 'r') as ifile:
    lines = ifile.readlines()
for line in lines[1:]:
    items = line.split(',')
    latlons[int(items[0])] = (float(items[1]), float(items[2]))

In [ ]:

def d2j(datestring):
    # Date to Julian date
    d = datetime.date.fromisoformat(datestring)
    jday = d.timetuple().tm_yday
    if calendar.isleap(d.year) and jday > 59:
        jday -= 1
    return jday

In [60]:
all_latlons = gpd.GeoDataFrame({'locnum': list(latlons.keys()), 'geometry': [shapely.Point(i[1], i[0]) for i in latlons.values()]})
centroid_point = all_latlons.dissolve().centroid
centroid_latlon = (centroid_point.y[0], centroid_point.x[0])

In [64]:
%%time
best_models = {varname: get_bestmodels(varname, centroid_latlon) for varname in ['tas', 'tasmax', 'tasmin', 'pr']}

with open('bestmodels.csv', 'a') as ofile:
    for varname in best_models:
        ofile.write('{0},{1},{2},{3},{4},{5}\n'.format(varname, *best_models[varname]))

CPU times: total: 8.25 s
Wall time: 20min 35s


In [67]:
calibration_fxns = {}
for varname in ['tas', 'tasmax', 'tasmin', 'pr']:
    calibration_fxns[varname] = {}
    for model in best_models[varname]:
        hist_mod = get_var(varname, model, centroid_latlon, yearshift = flip_year)
        hist_obs = get_var(varname, 'ERA5', centroid_latlon)
        o_quarters = quarters(hist_obs, HIST_START, HIST_END)
        m_quarters = quarters(hist_mod, HIST_START, HIST_END)
        calibration_fxns[varname][model] = [calibration_function(o_quarters[i].flatten(), m_quarters[i].flatten()) for i in range(4)]

In [76]:
with open('calib_fxns.csv', 'w') as ofile:
    for varname in calibration_fxns:
        for model in calibration_fxns[varname]:
            for season in range(4):
                ofile.write('{0},{1},{2},{3}\n'.format(varname, model, season+1, ','.join([str(i) for i in list(calibration_fxns[varname][model][season])])))

In [151]:
class Hazard:
    def get_percentile(self, latlon, q):
        southern_hem = int(latlon[0] < 0)
        era_data = {}
        scenario_years = {}
        varnames = self.varname.split('+')
        for varname in varnames:
            era_data[varname] = get_observed_gee(varname, latlon, PERCENTILE_STARTYEAR, PERCENTILE_ENDYEAR, southern_hem)
        countdist = self.val_dist([ed for ed in era_data])
        return np.percentile(countdist, q)
    
    def get_expectedval(self, latlon, dataset, calib_fxns, start_year, end_year):
        # Take uncalibrated data, calibrate it, apply val_dist() to calibrated data, use resulting freq dist
        # to parameterize Dirichlet prior, take resulting vector to parameterize multinomial distribution,
        # sample from that multinomial to generate predictive distribution of freq distributions, and return statistics
        # (mean and stdev but it could be anything) of the predictive distribution.
        
        southern_hem = int(latlon[0] < 0)
        
        numbins = end_year - start_year + 1
        
        calib_data = np.array(calibrate(dataset, calib_fxns))
        countdist = self.val_dist([calib_data[[0,152][int(not southern_hem)]:[len(calib_data),-213][int(not southern_hem)]]])
        if countdist is None:
            para_res[modelplus] = None
        else:
            observed_vals = np.array(list(countdist.keys()))
            cdist = {}
            minval = observed_vals[0]
            maxval = observed_vals[-1]
            D = (maxval - minval) / (numbins - 1)
            for i in range(numbins):
                centerval = minval + (i * D)
                cdist[centerval] = 0
            for count in countdist:
                for centerval in cdist:
                    if count >= centerval - (D/2) and count < centerval + (D/2):
                        cdist[centerval] += 1
            alpha = np.array(list(cdist.values())) + (1/numbins)
            res = []
            for i in range(10000):
                dirich_samp = np.random.dirichlet(alpha, 1)
                mult_samp = np.random.multinomial(end_year - start_year + 1, dirich_samp[0], 1)[0]
                res.append(sum([list(cdist.keys())[j] * mult_samp[j] for j in range(len(list(cdist.keys())))]) / (end_year - start_year + 1))
            res = np.array(res)

        result = {}
        if res is None:
            result = [-9999, -9999, -9999]
        else:
            result = [np.mean(res), np.std(res), -9999]  # -9999 is dummy to preserve format from earier version
        return result
    

class ThresholdDays(Hazard):
    # Num days (can be nonconsecutive) meeting some criterion in a year
    def __init__(self, hazname, varname, var_threshold, want_max):
        self.hazname = hazname
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_max = want_max
        self.probmodel = 'binomial'
        self.exceed_is_gte = True

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        
        if self.want_max:
            vals = np.sum(byyear >= self.var_threshold, axis=1)
        else:
            vals = np.sum(byyear <= self.var_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    
class ThresholdDaysSeasonal(Hazard):
    # Num days (can be nonconsecutive) meeting some criterion in a year
    def __init__(self, hazname, varname, var_threshold, want_max, startdate, enddate):
        self.hazname = hazname
        self.varname = varname
        self.var_threshold = var_threshold
        self.want_max = want_max
        self.startdate = startdate
        self.enddate = enddate
        self.probmodel = 'binomial'
        self.exceed_is_gte = True

    def val_dist(self, datalist):
        data = datalist[0]
        if data.size % 365 != 0:
            raise Exception('Data array length is not an integer multiple of 365')   
        byyear = data.reshape(data.size // 365, 365)
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array([inseason_onerow]*(data.size//365))
        if self.want_max:
            vals = np.sum(inseason >= self.var_threshold, axis=1)
        else:
            vals = np.sum(inseason <= self.var_threshold, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist
    

class AnnualVal(Hazard):
    def __init__(self, hazname, varname, aggtype):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        return result_dist

class SeasonalVal(Hazard):
    # Sum, mean, max, or min of some variable value within a year
    def __init__(self, hazname, varname, aggtype, startdate, enddate, southern_hem):
        self.hazname = hazname
        self.varname = varname
        self.aggtype = aggtype
        self.startdate = startdate
        self.enddate = enddate
        self.southern_hem = southern_hem
        self.probmodel = 'binomial'
        self.exceed_is_gte = True
        
    def val_dist(self, datalist):
        data = datalist[0]
        byyear = data.reshape(data.size//365, 365)
        start_jday = d2j('1999-{0}'.format(self.startdate)) - [0, 182][int(self.southern_hem)]
        end_jday = d2j('1999-{0}'.format(self.enddate)) - [0, 182][int(self.southern_hem)]
        if end_jday < start_jday:
            end_jday += 365
        inseason_onerow = [((i >= start_jday)and(i <= end_jday)) for i in range(365)]
        inseason = np.array([inseason_onerow]*(data.size//365))
        byyear = byyear * inseason
        if self.aggtype == 'sum':
            vals = np.sum(byyear, axis=1)
        elif self.aggtype == 'mean':
            vals = np.mean(byyear, axis=1)
        elif self.aggtype == 'max':
            vals = np.max(byyear, axis=1)
        elif self.aggtype == 'min':
            vals = np.min(byyear, axis=1)
        result_dist = {}
        for val in np.unique(vals):
            result_dist[val] = np.sum(vals == val)
        #print(vals)
        return result_dist

In [195]:
SEASONS = {
    'SW-monsoon': {'start': '06-01', 'end': '09-30'},
    'NE-monsoon': {'start': '11-01', 'end': '12-31'}
}

FUTURE_INTERVALS = [
    {'start': 1995, 'end': 2004},
    {'start': 2005, 'end': 2014},
    {'start': 2020, 'end': 2029},
    {'start': 2030, 'end': 2039},
    {'start': 2040, 'end': 2049},
    {'start': 2050, 'end': 2059},
    {'start': 2060, 'end': 2069},
    {'start': 2070, 'end': 2079},
    {'start': 2080, 'end': 2089},
    {'start': 2090, 'end': 2099}
]

SCENARIOS = ['ssp245', 'ssp585']

HAZARDS = [
    SeasonalVal('mean maxtemp SWMonsoon', 'tasmax', 'mean', SEASONS['SW-monsoon']['start'], SEASONS['SW-monsoon']['end'], False), # Mean seasonal Maximum temperature
    SeasonalVal('mean maxtemp NEMonsoon', 'tasmax', 'mean', SEASONS['NE-monsoon']['start'], SEASONS['NE-monsoon']['end'], False),
    SeasonalVal('mean mintemp SWMonsoon', 'tasmin', 'mean', SEASONS['SW-monsoon']['start'], SEASONS['SW-monsoon']['end'], False), # Mean seasonal Minimum temperature
    SeasonalVal('mean mintemp NEMonsoon', 'tasmin', 'mean', SEASONS['NE-monsoon']['start'], SEASONS['NE-monsoon']['end'], False),
    SeasonalVal('total precip SWMonsoon', 'pr', 'mean', SEASONS['SW-monsoon']['start'], SEASONS['SW-monsoon']['end'], False), # Seasonal Cumulative Rainfall
    SeasonalVal('total precip NEMonsoon', 'pr', 'mean', SEASONS['NE-monsoon']['start'], SEASONS['NE-monsoon']['end'], False),
    ThresholdDaysSeasonal('days precip-gte-0 SWMonsoon', 'pr', 0.001, True, SEASONS['SW-monsoon']['start'], SEASONS['SW-monsoon']['end']), # Seasonal number of rainy days
    ThresholdDaysSeasonal('days precip-gte-0 NEMonsoon', 'pr', 0.001, True, SEASONS['NE-monsoon']['start'], SEASONS['NE-monsoon']['end']),
    AnnualVal('mean meantemp', 'tas', 'mean'), # Projected Change in Annual Average Temperature
    ThresholdDays('days precip-gte-10', 'pr', 10, True), # Projected Change in Extreme Precipitation Days
    AnnualVal('mean mintemp', 'tasmin', 'mean'), # Projected Change in Annual Average Minimum Temperature
    AnnualVal('mean maxtemp', 'tasmax', 'mean'), # Projected Change in Annual Average Maximum Temperature
    AnnualVal('total precip', 'pr', 'sum'), # Projected Change in Cumulative Precipitation
]

In [93]:
total_lats = []
total_lons = []
with open('total_latlons.csv', 'r') as ifile:
    lines = ifile.readlines()
for line in lines:
    items = [i.strip() for i in line.split(',')]
    if items[0]:
        total_lats.append(float(items[0]))
    if items[1]:
        total_lons.append(float(items[1]))

In [206]:
def do_locationhazard(hazard, dataset, calib_fxns, start_year, end_year):
    return hazard.get_expectedval(latlon, dataset, calib_fxns, start_year, end_year)[0]

In [ ]:
for hazard in HAZARDS:
    print(hazard.hazname)
    for scenario_idx, scenario in enumerate(SCENARIOS):
        print(scenario)
        for future_interval in FUTURE_INTERVALS:
            if future_interval['end'] >= 2015 or scenario_idx == 0:
                print("  {0}-{1}".format(future_interval['start'], future_interval['end']))
                num_done = 0
                means = []
                stds = []
                for lat_idx, lat in enumerate(total_lats):
                    for lon_idx, lon in enumerate(total_lons):
                        latlon = (lat, lon)
                        if latlon in list(latlons.values()):
                            result = []
                            for model in best_models[hazard.varname]:
                                dataset = get_var(hazard.varname, model, latlon, start_year=future_interval['start'], end_year=future_interval['end'], yearshift=False, scenario=scenario)
                                calib_fxns = calibration_fxns[hazard.varname][model]
                                result.append(do_locationhazard(hazard, dataset, calib_fxns, start_year=future_interval['start'], end_year=future_interval['end']))
                            result = np.array(result)
                            means.append(result.mean())
                            stds.append(result.std())
                            num_done += 1
                            print(num_done, end=' ')
                        else:
                            means.append(None)
                            stds.append(None)
                print()
                outarray_mean = np.array(means).reshape(len(total_lats), len(total_lons))
                outarray_std = np.array(stds).reshape(len(total_lats), len(total_lons))
                xarray_mean= xr.DataArray(outarray_mean, {'latitude': total_lats, 'longitude': total_lons})
                xarray_std= xr.DataArray(outarray_std, {'latitude': total_lats, 'longitude': total_lons})
                xarray_mean.to_raster('{0}__{1}_{2}-{3}_mean.tif'.format(hazard.hazname.replace(' ', '_'), scenario, future_interval['start'], future_interval['end']))
                xarray_std.to_raster('{0}__{1}_{2}-{3}_std.tif'.format(hazard.hazname.replace(' ', '_'), scenario, future_interval['start'], future_interval['end']))

mean maxtemp SWMonsoon
ssp245
  1995-2004
1 2 3 4 5 